# Thai Classical Music Motif Analysis – Proof of Concept (Desc)





## 1. Data Loading & Corpus Overview
- Load JSON / MIDI-derived symbolic data  
- Basic stats (songs per motif, bars per song)

## 2. Symbolic Normalization
- Note vocabulary definition
- Rest handling
- Octave / ornament normalization
- Sequence flattening (bar → token stream)

## 3. Feature Extraction
### 3.1 Note-Level Statistics
- Pitch histogram
- Interval distribution

### 3.2 N-gram Extraction
- Unigram / Bigram / Trigram counts
- Per-motif aggregation

### 3.3 Contour Representation (optional)
- Up / Down / Same encoding
- Contour n-grams

## 4. Motif Similarity Analysis
- Feature vector construction
- Cosine / KL similarity
- Song-to-song and motif-level similarity

## 5. Motif Classification (Baseline)
- Train / validation split
- Linear models (LogReg / SVM)
- Confusion matrix & accuracy

## 6. Visualization
- N-gram heatmaps
- PCA / UMAP motif clustering

## 7. Discussion (Preliminary Findings)
- Observed motif patterns
- Limitations (small data, single layer)
- Implications for Stage 2 tagging

## 8. Next Steps
- Dataset expansion
- Audio features (MFCC / chroma)
- Conditional generation pipeline


#


# Thai Classical Music Motif Analysis – Code


## 0. Environment Setup (once per runtime)




In [ ]:
!pip install mido python-rtmidi

In [ ]:
!wget https://github.com/Phonbopit/sarabun-webfont/raw/master/fonts/thsarabunnew-webfont.ttf


In [ ]:
!git clone https://github.com/GetomG/Thai-Music-Thesis.git
# If already cloned:
# !git pull

In [ ]:
# Enter project folder
%cd Thai-Music-Thesis

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.font_manager.fontManager.addfont('thsarabunnew-webfont.ttf')
mpl.rc('font', family='TH Sarabun New')
mpl.rcParams["axes.unicode_minus"] = False

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Helpers Libs


In [ ]:
# ============================================================
# 2. I/O Utilities
# ============================================================
from thai_music_utils.io_utils import (
    save_json_bar_per_line
)

# ============================================================
# 3. Notation Processing
# - Flatten JSON → tokens
# - Normalize octave markers
# - Convert to continuous sequence string
# ============================================================
from thai_music_utils.notation_utils import (
    flatten_song_notation,
    normalize_octave_markers,
    notation_to_sequence
)

# ============================================================
# 4. Octave Inference (DP-based register guessing)
# ============================================================
from thai_music_utils.octave_inference import (
    is_thai_note,
    get_fixed_octave,
    guess_octaves_with_constraints,
    add_octaves_respecting_labels
)

# ============================================================
# 5. Preprocessing Utilities
# ============================================================
from thai_music_utils.preprocessing import (
    flatten_song_data,
    remove_all_signs
)

# ============================================================
# 6. MIDI Rendering (Ranad-specific)
# ============================================================
from thai_music_utils.midi_ranad import (
    generate_ranad_midi
)

# ============================================================
# 7. EDA Helpers (Symbolic Analysis)
# ============================================================
from thai_music_utils.eda_symbolic_normalization import (
    normalize_token,
    normalize_bar,
    flatten_song,
    THAI_NOTES,
    UP_MARK,
    LOW_MARK,
    REST_TOKEN
)

from thai_music_utils.eda_stats import (
    extract_symbols,
    pitch_stats,
    stats_to_df
)

## 1. Data Loading & Corpus Overview
- Load JSON / MIDI-derived symbolic data  
- Basic stats (songs per motif, bars per song)


In [ ]:
import json
from pathlib import Path
from collections import defaultdict

BASE = Path("/content/drive/MyDrive/thai_music_data/songs")

songs = []

for motif_dir in BASE.iterdir():
    if not motif_dir.is_dir():
        continue

    motif = motif_dir.name

    for song_dir in motif_dir.iterdir():
        json_dir = song_dir / "json"
        if not json_dir.exists():
            continue

        for json_file in json_dir.glob("*.json"):
            try:
                with open(json_file, "r", encoding="utf-8") as f:
                    data = json.load(f)

                songs.append({
                    "motif": motif,
                    "song": song_dir.name,
                    "path": str(json_file),
                    "data": data
                })

            except Exception as e:
                print(f"⚠️ Skipped {json_file}: {e}")

In [ ]:
print(f"Total songs loaded: {len(songs)}")

by_motif = defaultdict(int)
for s in songs:
    by_motif[s["motif"]] += 1

for motif, n in by_motif.items():
    print(f"{motif}: {n} songs")

In [ ]:
from pathlib import Path

BASE = Path("/content/drive/MyDrive/thai_music_data/songs")

for motif_dir in BASE.iterdir():
    if not motif_dir.is_dir():
        continue

    for song_dir in motif_dir.iterdir():
        json_dir = song_dir / "json"
        if not json_dir.exists():
            continue

        json_files = sorted(p.name for p in json_dir.glob("*.json"))
        if not json_files:
            continue

        print(f"{motif_dir.name}/{song_dir.name}:")
        for jf in json_files:
            print(f"  - {jf}")

## 2. Symbolic Normalization (preprocessing)
- Note vocabulary definition
- Rest handling
- Octave / ornament normalization
- Sequence flattening (bar → token stream)


In [ ]:
# ----------------------------
# Apply normalization to corpus
# ----------------------------
for s in songs:
    try:
        # Store normalized symbolic sequence WITHOUT overwriting raw data
        s["sequence"] = flatten_song(s["data"])
    except Exception as e:
        # Fault-tolerant: broken JSON → empty sequence
        print(f"⚠️ Normalization failed for {s['song']}: {e}")
        s["sequence"] = []


# ----------------------------
# Sanity check (inspect output)
# ----------------------------
songs[29]["sequence"][:40]

## 3. Feature Extraction



## 3.1 Note-Level Statistics
- Pitch histogram
- Interval distribution


### 3.1.1 Pitch / Symbol Histogram
We want to know which notes are used and how often.
- Count how often ด, ดํ, ซฺ, etc. appear
#### This tells us:
- register preference (high / low)
- stylistic bias per motif
- We split compound tokens like ซลดํล into individual notes
- Rests are ignored


In [ ]:
# With octave information
pitch_with_octave = pitch_stats(songs, strip_octave=False)
df_pitch_with_octave = stats_to_df(pitch_with_octave)

# Without octave information
pitch_no_octave = pitch_stats(songs, strip_octave=True)
df_pitch_no_octave = stats_to_df(pitch_no_octave)

In [ ]:
df_pitch_no_octave["ลาว"]

In [ ]:
df_pitch_no_octave["แขก"]

In [ ]:
df_pitch_no_octave["เขมร"]

In [ ]:
df_pitch_no_octave["พม่า"]

### 3.1.2 Interval Distribution


In [ ]:
PITCH_ORDER = {
    "ซฺ": -3, "ลฺ": -2, "ทฺ": -1,
    "ด": 0, "ร": 1, "ม": 2, "ฟ": 3, "ซ": 4, "ล": 5, "ท": 6,
    "ดํ": 7, "รํ": 8, "มํ": 9, "ฟํ": 10, "ซํ": 11, "ลํ": 12,
}

In [ ]:
def interval_bins(intervals):
    bins = {
        "Down step (-1)": 0,
        "Same (0)": 0,
        "Up step (+1)": 0,
        "Medium jump (2–3)": 0,
        "Large leap (≥4)": 0
    }

    for iv in intervals:
        if iv == -1:
            bins["Down step (-1)"] += 1
        elif iv == 0:
            bins["Same (0)"] += 1
        elif iv == 1:
            bins["Up step (+1)"] += 1
        elif abs(iv) in (2, 3):
            bins["Medium jump (2–3)"] += 1
        elif abs(iv) >= 4:
            bins["Large leap (≥4)"] += 1

    return bins

In [ ]:
def extract_intervals(sequence):
    """
    Convert a symbolic sequence into signed pitch intervals
    using the PITCH_ORDER dict.
    """
    symbols = extract_symbols(sequence)
    intervals = []

    for a, b in zip(symbols, symbols[1:]):
        if a in PITCH_ORDER and b in PITCH_ORDER:
            intervals.append(PITCH_ORDER[b] - PITCH_ORDER[a])

    return intervals

In [ ]:
interval_bin_pct_by_motif = {}

for motif in {s["motif"] for s in songs}:
    all_intervals = []
    for s in songs:
        if s["motif"] == motif:
            all_intervals.extend(extract_intervals(s["sequence"]))

    bins = interval_bins(all_intervals)
    total = sum(bins.values())

    interval_bin_pct_by_motif[motif] = {
        k: (v / total * 100 if total else 0)
        for k, v in bins.items()
    }

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_grouped_vertical(interval_bin_pct_by_motif, title):

    motifs = list(interval_bin_pct_by_motif.keys())
    bins = list(next(iter(interval_bin_pct_by_motif.values())).keys())

    n_bins = len(bins)
    n_motifs = len(motifs)

    bar_width = 0.12   # 🔥 MUCH thinner
    x = np.arange(n_bins) * 1.1   # 🔥 add spacing between groups

    colors = plt.get_cmap("Set3").colors  # cleaner palette

    plt.figure(figsize=(14, 6))  # 🔥 wider figure

    for i, motif in enumerate(motifs):
        values = [interval_bin_pct_by_motif[motif][b] for b in bins]

        plt.bar(
            x + i * bar_width,
            values,
            width=bar_width,
            label=motif,
            color=colors[i],
            edgecolor="black",
            linewidth=0.4
        )

    plt.xticks(
        x + bar_width * (n_motifs - 1) / 2,
        bins,
        rotation=30
    )

    plt.ylabel("Percentage (%)")
    plt.title(title)
    plt.legend(frameon=False)
    plt.grid(axis="y", linestyle="--", alpha=0.3)

    plt.tight_layout()
    plt.show()

In [ ]:
plot_grouped_vertical(
    interval_bin_pct_by_motif,
    "Interval Jump Distribution (%) – Motif Comparison"
)

In [ ]:
import pandas as pd

rows = []

for motif, pct_bins in interval_bin_pct_by_motif.items():
    rows.append({
        "motif": motif,
        "stepwise_%": (
            pct_bins["Down step (-1)"]
            + pct_bins["Same (0)"]
            + pct_bins["Up step (+1)"]
        ),
        "medium_%": pct_bins["Medium jump (2–3)"],
        "large_%": pct_bins["Large leap (≥4)"],
    })

df_interval_bins = (
    pd.DataFrame(rows)
    .sort_values("motif")
    .reset_index(drop=True)
)

df_interval_bins = df_interval_bins.round(2)

df_interval_bins

## 3.2 N-grams


### 3.2.1 Symbol sequence construction
- We take each song and turn it into one linear sequence of symbols that n-grams can operate on.

- Future work may incorporate rest-aware segmentation, where extended rests are treated as phrase boundaries rather than melodic tokens.


In [ ]:
def song_to_symbol_sequence(song, strip_octave=True):
    """
    Convert a song dict into a flat sequence of note symbols.
    """
    sequence = song["sequence"]  # already flattened bars
    symbols = extract_symbols(sequence, strip_octave=strip_octave)
    return symbols

In [ ]:
s = songs[0]
seq = song_to_symbol_sequence(s)
seq[:30]

In [ ]:
for s in songs:
    s["symbol_sequence"] = song_to_symbol_sequence(s)

In [ ]:
songs[0]["motif"], len(songs[0]["symbol_sequence"])

### 3.2.2 Unigram Statistics (same as hist above)

- probably the same as histogram above

- This provides a baseline representation of melodic vocabulary and serves as a consistency check before moving to higher-order n-grams.


### 3.2.3 Bigram Statistics

- Bigram statistics count the frequency of consecutive pairs of note symbols in the symbolic sequence.

- This captures local melodic transition patterns and provides insight into motif-specific note-to-note grammar beyond individual pitch usage.


In [ ]:
from collections import Counter

def extract_bigrams(symbol_sequence):
    """
    Convert a symbol sequence into bigrams.
    Example: ['ด','ร','ม'] -> [('ด','ร'), ('ร','ม')]
    """
    return zip(symbol_sequence, symbol_sequence[1:])

In [ ]:
bigram_counts_by_motif = {}

for motif in {s["motif"] for s in songs}:
    counter = Counter()

    for s in songs:
        if s["motif"] == motif:
            counter.update(extract_bigrams(s["symbol_sequence"]))

    bigram_counts_by_motif[motif] = counter

In [ ]:
import pandas as pd

df_bigram_by_motif = {}

for motif, counter in bigram_counts_by_motif.items():
    total = sum(counter.values())

    df = (
        pd.DataFrame([
            {
                "bigram": f"{a}→{b}",
                "count": cnt,
                "percent": cnt / total * 100
            }
            for (a, b), cnt in counter.items()
        ])
        .sort_values("count", ascending=False)
        .reset_index(drop=True)
    )

    df_bigram_by_motif[motif] = df

In [ ]:
def merge_topk_side_by_side(df_bigram_by_motif, top_k=15):

    merged = pd.DataFrame()

    for motif, df in df_bigram_by_motif.items():
        top = df.head(top_k).copy()
        top = top[["bigram", "percent"]]

        # round to 2 decimal places
        top["percent"] = top["percent"].round(2)

        top.columns = [
            f"{motif}_bigram",
            f"{motif}_%"
        ]

        merged = pd.concat([merged, top], axis=1)

    return merged


merged_df = merge_topk_side_by_side(df_bigram_by_motif, top_k=15)
merged_df

### 3.2.4 Trigram Statistics

- Trigram statistics count the frequency of three consecutive note symbols in the symbolic sequence.

- Trigrams capture short melodic phrases and local contour patterns, which are often more distinctive of motif style than single notes or pairwise transitions.


In [ ]:
from collections import Counter

def extract_trigrams(symbol_sequence):
    """
    Convert a symbol sequence into trigrams.
    Example: ['ด','ร','ม','ซ'] -> [('ด','ร','ม'), ('ร','ม','ซ')]
    """
    return zip(symbol_sequence, symbol_sequence[1:], symbol_sequence[2:])

In [ ]:
trigram_counts_by_motif = {}

for motif in {s["motif"] for s in songs}:
    counter = Counter()

    for s in songs:
        if s["motif"] == motif:
            counter.update(extract_trigrams(s["symbol_sequence"]))

    trigram_counts_by_motif[motif] = counter

In [ ]:
import pandas as pd

df_trigram_by_motif = {}

for motif, counter in trigram_counts_by_motif.items():
    total = sum(counter.values())

    df = (
        pd.DataFrame([
            {
                "trigram": f"{a}→{b}→{c}",
                "count": cnt,
                "percent": cnt / total * 100
            }
            for (a, b, c), cnt in counter.items()
        ])
        .sort_values("count", ascending=False)
        .reset_index(drop=True)
    )

    df_trigram_by_motif[motif] = df

In [ ]:
def merge_topk_trigram_side_by_side(df_trigram_by_motif, top_k=15):

    merged = pd.DataFrame()

    for motif, df in df_trigram_by_motif.items():
        top = df.head(top_k).copy()
        top = top[["trigram", "percent"]]

        # round to 2 decimals
        top["percent"] = top["percent"].round(2)

        top.columns = [
            f"{motif}_trigram",
            f"{motif}_%"
        ]

        merged = pd.concat([merged, top], axis=1)

    return merged


merged_trigram_df = merge_topk_trigram_side_by_side(df_trigram_by_motif, top_k=15)
merged_trigram_df

### 3.2.5 Quadgram Statistics


In [ ]:
from collections import Counter

def extract_quadgrams(symbol_sequence):
    """
    Convert a symbol sequence into quadgrams.
    Example: ['ด','ร','ม','ซ','ล']
    -> [('ด','ร','ม','ซ'), ('ร','ม','ซ','ล')]
    """
    return zip(
        symbol_sequence,
        symbol_sequence[1:],
        symbol_sequence[2:],
        symbol_sequence[3:]
    )

In [ ]:
quadgram_counts_by_motif = {}

for motif in {s["motif"] for s in songs}:
    counter = Counter()

    for s in songs:
        if s["motif"] == motif:
            counter.update(extract_quadgrams(s["symbol_sequence"]))

    quadgram_counts_by_motif[motif] = counter

In [ ]:
import pandas as pd

df_quadgram_by_motif = {}

for motif, counter in quadgram_counts_by_motif.items():
    total = sum(counter.values())

    df = (
        pd.DataFrame([
            {
                "quadgram": f"{a}→{b}→{c}→{d}",
                "count": cnt,
                "percent": cnt / total * 100
            }
            for (a, b, c, d), cnt in counter.items()
        ])
        .sort_values("count", ascending=False)
        .reset_index(drop=True)
    )

    df_quadgram_by_motif[motif] = df

In [ ]:
def merge_topk_quadgram_side_by_side(df_quadgram_by_motif, top_k=15):

    merged = pd.DataFrame()

    for motif, df in df_quadgram_by_motif.items():
        top = df.head(top_k).copy()
        top = top[["quadgram", "percent"]]

        top["percent"] = top["percent"].round(2)

        top.columns = [
            f"{motif}_quadgram",
            f"{motif}_%"
        ]

        merged = pd.concat([merged, top], axis=1)

    return merged


merged_quad_df = merge_topk_quadgram_side_by_side(df_quadgram_by_motif, top_k=15)
merged_quad_df

## 4. Motif Similarity Analysis – Feature Vector


### 4.1 Feature Vector 𝒛 construction

Each motif is represented by a fixed-dimensional feature vector 𝒛 that summarizes its symbolic melodic characteristics.

- The first part of 𝒛 encodes the octave-agnostic pitch distribution (ด, ร, ม, ฟ, ซ, ล, ท), capturing melodic vocabulary preferences.
- The second part of 𝒛 encodes interval magnitude statistics (stepwise, medium, large), capturing characteristic melodic motion patterns.

This compact representation enables direct quantitative comparison between motifs using standard similarity measures.


	1.	Directional interval stats (up / down / same)  → +3D
	2.	Interval entropy → +1D
	3.	Pitch-class entropy → +1D
	4.	Top-K pitch bigram frequencies → +K D (we’ll keep K small, e.g. 5–10)



In [ ]:
PITCH_ORDER = ["ด", "ร", "ม", "ฟ", "ซ", "ล", "ท"]

In [ ]:
# build pitch vector per motif (7D)

def pitch_vector_from_df(df):
    """
    Convert pitch DataFrame into a fixed-order vector.
    Missing notes get 0.
    """
    pitch_map = dict(zip(df["note"], df["percent"]))
    return [pitch_map.get(n, 0.0) for n in PITCH_ORDER]

# build interval vector per motif (3D)
def interval_vector_from_df(df_interval_bins, motif):
    row = df_interval_bins[df_interval_bins["motif"] == motif].iloc[0]
    return [
        row["stepwise_%"],
        row["medium_%"],
        row["large_%"]
    ]

In [ ]:
import numpy as np

motif_vectors = {}

for motif in df_interval_bins["motif"]:
    pitch_vec = pitch_vector_from_df(df_pitch_no_octave[motif])
    interval_vec = interval_vector_from_df(df_interval_bins, motif)

    motif_vectors[motif] = np.array(pitch_vec + interval_vec)

*Each* motif is now a point in 10D space:

	- dimensions 1–7 → what notes are used
	- dimensions 8–10 → how melodies move

Similarity = closeness in this space.


In [ ]:
motif_vectors["ลาว"], motif_vectors["ลาว"].shape

## 4.2 Motif-level Similarity (Cosine Similarity)

- To quantify similarity between motifs, cosine similarity is computed between their feature vectors.
Cosine similarity measures the angular similarity between vectors, focusing on distributional shape rather than absolute magnitude.

- This is well suited for percentage-based symbolic features and provides an interpretable measure of stylistic closeness between motifs.


In [ ]:
#remove ไทยเดิม as of now data not complete

# Remove motifs with insufficient data
motif_vectors = {
    m: v for m, v in motif_vectors.items()
    if m not in {"ไทยเดิม", "พม่า"}
}

In [ ]:
import numpy as np
import pandas as pd

motifs = list(motif_vectors.keys())
X = np.vstack([motif_vectors[m] for m in motifs])

X.shape   # (num_motifs, 10)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

sim_matrix = cosine_similarity(X)

df_similarity = pd.DataFrame(
    sim_matrix,
    index=motifs,
    columns=motifs
)

df_similarity

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(6, 5))
plt.imshow(df_similarity.values, cmap="viridis", vmin=0.9, vmax=1.0)
plt.colorbar(label="Cosine Similarity")

plt.xticks(range(len(df_similarity.columns)), df_similarity.columns, rotation=45)
plt.yticks(range(len(df_similarity.index)), df_similarity.index)

plt.title("Motif-level Cosine Similarity")
plt.tight_layout()
plt.show()

At this stage, each motif is represented by a single 10-dimensional vector obtained by aggregating symbolic statistics across all songs of that motif.


### Feature vector V2 (13D)
	1.	Directional interval stats (up / down / same)  → +3D


In [ ]:
# Numeric ordering (for intervals / direction)
PITCH_TO_INT = {
    "ด": 0, "ร": 1, "ม": 2, "ฟ": 3, "ซ": 4, "ล": 5, "ท": 6,
}

# Just for display / plotting
PITCH_LIST = ["ด", "ร", "ม", "ฟ", "ซ", "ล", "ท"]

In [ ]:
def directional_interval_stats(symbols):
    """
    Compute directional interval percentages: up / down / same.
    """
    up = down = same = 0

    for a, b in zip(symbols, symbols[1:]):
        if a not in PITCH_ORDER or b not in PITCH_ORDER:
            continue

        ia = PITCH_TO_INT[a]
        ib = PITCH_TO_INT[b]

        if ib > ia:
            up += 1
        elif ib < ia:
            down += 1
        else:
            same += 1

    total = up + down + same
    if total == 0:
        return [0.0, 0.0, 0.0]

    return [
        up / total * 100,
        down / total * 100,
        same / total * 100
    ]

def motif_directional_vector(songs, motif):
    all_symbols = []
    for s in songs:
        if s["motif"] == motif:
            all_symbols.extend(s["symbol_sequence"])

    return directional_interval_stats(all_symbols)


In [ ]:
motif_vectors_v2 = {}

for motif, base_vec in motif_vectors.items():
    dir_vec = motif_directional_vector(songs, motif)
    motif_vectors_v2[motif] = np.concatenate([base_vec, dir_vec])

In [ ]:
motif_vectors_v2["ลาว"].shape
# should now be (13,)

## 4.x fragment-to-motif demo


In [ ]:
def get_fragment_symbols(song, n_symbols=16):
    """
    Extract a short fragment (approx. 2 bars) from a song.
    """
    return song["symbol_sequence"][:n_symbols]


def pitch_vector_from_symbols(symbols):
    from collections import Counter

    counter = Counter(symbols)
    total = sum(counter.values())

    return [
        counter.get(n, 0) / total * 100 if total else 0
        for n in PITCH_ORDER
    ]

def interval_vector_from_symbols(symbols):
    intervals = []
    for a, b in zip(symbols, symbols[1:]):
        if a in PITCH_ORDER and b in PITCH_ORDER:
            intervals.append(PITCH_ORDER.index(b) - PITCH_ORDER.index(a))

    bins = {"stepwise": 0, "medium": 0, "large": 0}

    for iv in intervals:
        a = abs(iv)
        if a <= 1:
            bins["stepwise"] += 1
        elif a <= 3:
            bins["medium"] += 1
        else:
            bins["large"] += 1

    total = sum(bins.values())
    return [
        bins["stepwise"] / total * 100 if total else 0,
        bins["medium"] / total * 100 if total else 0,
        bins["large"] / total * 100 if total else 0,
    ]

import numpy as np

def fragment_vector(symbols):
    return np.array(
        pitch_vector_from_symbols(symbols)
        + interval_vector_from_symbols(symbols)
    )

In [ ]:
def fragment_vector_v2(symbols):
    # v1 features
    pitch_vec = pitch_vector_from_symbols(symbols)
    interval_vec = interval_vector_from_symbols(symbols)

    # new v2 feature
    dir_vec = directional_interval_stats(symbols)

    return np.concatenate([pitch_vec, interval_vec, dir_vec])  # 13D

### Play around here (only v1 and v2?)


In [ ]:
raw_frag = [ "--ดํล", "ดํลรํดํ", "--ฟร", "ฟรซฟ", "--ดํล", "--รํดํ", "ฟรซฟ", "ลซดํล" ] #khmer เขมร
frag = extract_symbols(raw_frag, strip_octave=True)

In [ ]:
raw_frag = [ "ดํรํดํล", "ดํซลดํ", "ดํรํดํล", "ดํลซฟ", "----", "ดรฟซ", "ลฟลซ", "-ฟ-ร" ] #khmer เขมร
frag = extract_symbols(raw_frag, strip_octave=True)


In [ ]:
raw_frag = [ "----", "----", "-ซ-ล", "-ท-ดํ", "---ท", "--ดํรํ", "มํรํดํท", "ดํรํ-ดํ" ] #แขก
frag = extract_symbols(raw_frag, strip_octave=True)

In [ ]:
raw_frag = [ "----", "----", "--ซล", "ดํมรด", "รม-ซ", "-ล--", "ดํลซม", "ซล-ดํ","รํมํรํดํ" ] #ลาว
frag = extract_symbols(raw_frag, strip_octave=True)

In [ ]:
raw_frag = [
          "----",
          "รมซล",
          "ซลดํล",
          "-ซซซ",
          "-ดรม",
          "-ซ-ล",
          "-ดํ-มํ",
          "-รํมํซํ"
        ] #ลาว
frag = extract_symbols(raw_frag, strip_octave=True)

In [ ]:
raw_frag =[
          "ดํลซม",
          "-ร-ด",
          "--มร",
          "ดร-ม",
          "รด-ร",
          "-ม-ซ",
          "ลซดํล",
          "ซม-ร"
        ] #ลาว

frag = extract_symbols(raw_frag, strip_octave=True)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

frag_vec = fragment_vector(frag)

scores = {}
for motif, vec in motif_vectors.items():
    scores[motif] = cosine_similarity(
        frag_vec.reshape(1, -1),
        vec.reshape(1, -1)
    )[0, 0]

sorted(scores.items(), key=lambda x: x[1], reverse=True)

In [ ]:
frag_vec = fragment_vector_v2(frag)

scores = {}
for motif, vec in motif_vectors_v2.items():
    scores[motif] = cosine_similarity(
        frag_vec.reshape(1, -1),
        vec.reshape(1, -1)
    )[0, 0]

sorted(scores.items(), key=lambda x: x[1], reverse=True)

## 4. Feature vector building for Motifs cleaned

Each motif is represented by a fixed-dimensional feature vector 𝒛 that summarizes its symbolic melodic characteristics.

- The first part of 𝒛 encodes the octave-agnostic pitch distribution (ด, ร, ม, ฟ, ซ, ล, ท), capturing melodic vocabulary preferences.
- The second part of 𝒛 encodes interval magnitude statistics (stepwise, medium, large), capturing characteristic melodic motion patterns.

This compact representation enables direct quantitative comparison between motifs using standard similarity measures.


	1.	Base 7D (notes) + Directional interval stats (up / down / same)  → +3D
	2.	Interval entropy → +1D
	3.	Pitch-class entropy → +1D
	4.	Top-K pitch bigram frequencies → +K D (we’ll keep K small, e.g. 5–10)


### Helpers for Vectors


In [ ]:
# ===== Pitch definitions =====
PITCH_ORDER = ["ด", "ร", "ม", "ฟ", "ซ", "ล", "ท"]

PITCH_TO_INT = {
    "ด": 0, "ร": 1, "ม": 2, "ฟ": 3, "ซ": 4, "ล": 5, "ท": 6,
}

# ===== Symbol helpers =====
from collections import Counter
import numpy as np


# Pitch statistics
def pitch_vector_from_symbols(symbols):
    counter = Counter(symbols)
    total = sum(counter.values())

    return [
        counter.get(n, 0) / total * 100 if total else 0
        for n in PITCH_ORDER
    ]

#Interval statistics (magnitude)
def interval_vector_from_symbols(symbols):
    intervals = []

    for a, b in zip(symbols, symbols[1:]):
        if a in PITCH_TO_INT and b in PITCH_TO_INT:
            intervals.append(PITCH_TO_INT[b] - PITCH_TO_INT[a])

    bins = {"stepwise": 0, "medium": 0, "large": 0}

    for iv in intervals:
        a = abs(iv)
        if a <= 1:
            bins["stepwise"] += 1
        elif a <= 3:
            bins["medium"] += 1
        else:
            bins["large"] += 1

    total = sum(bins.values())
    return [
        bins["stepwise"] / total * 100 if total else 0,
        bins["medium"] / total * 100 if total else 0,
        bins["large"] / total * 100 if total else 0,
    ]

#for V2
def directional_interval_stats(symbols):
    up = down = same = 0

    for a, b in zip(symbols, symbols[1:]):
        if a not in PITCH_TO_INT or b not in PITCH_TO_INT:
            continue

        ia = PITCH_TO_INT[a]
        ib = PITCH_TO_INT[b]

        if ib > ia:
            up += 1
        elif ib < ia:
            down += 1
        else:
            same += 1

    total = up + down + same
    if total == 0:
        return [0.0, 0.0, 0.0]

    return [
        up / total * 100,
        down / total * 100,
        same / total * 100,
    ]


#for V3
import math
from collections import Counter

def interval_entropy(symbols):
    """
    Shannon entropy of signed pitch intervals.
    """
    intervals = []
    for a, b in zip(symbols, symbols[1:]):
        if a in PITCH_TO_INT and b in PITCH_TO_INT:
            intervals.append(PITCH_TO_INT[b] - PITCH_TO_INT[a])

    if not intervals:
        return 0.0

    counter = Counter(intervals)
    total = sum(counter.values())

    entropy = 0.0
    for cnt in counter.values():
        p = cnt / total
        entropy -= p * math.log2(p)

    return entropy


# for V4
import math
from collections import Counter

def pitch_class_entropy(symbols):
    """
    Shannon entropy of pitch-class usage (octave-agnostic).
    """
    pitches = [s for s in symbols if s in PITCH_ORDER]
    if not pitches:
        return 0.0

    counter = Counter(pitches)
    total = sum(counter.values())

    entropy = 0.0
    for cnt in counter.values():
        p = cnt / total
        entropy -= p * math.log2(p)

    return entropy

#for V5
from collections import Counter

def extract_pitch_bigrams(symbols):
    """
    Extract pitch bigrams (order-sensitive, octave-agnostic).
    """
    pitches = [s for s in symbols if s in PITCH_ORDER]
    return list(zip(pitches, pitches[1:]))

def get_top_k_bigrams(songs, motifs, K=8):
    counter = Counter()

    for s in songs:
        if s["motif"] in motifs:
            bigrams = extract_pitch_bigrams(s["symbol_sequence"])
            counter.update(bigrams)

    return [bg for bg, _ in counter.most_common(K)]

### V1 (10D) BASE
  - **pitch** : dimensions 1–7 → what notes are used
  - **interval** : dimensions 8–10 → how melodies move


In [ ]:
def motif_vector_v1(songs, motif):
    all_symbols = []
    for s in songs:
        if s["motif"] == motif:
            all_symbols.extend(s["symbol_sequence"])

    pitch_vec = pitch_vector_from_symbols(all_symbols)          #7D
    interval_vec = interval_vector_from_symbols(all_symbols)    #3D

    return np.array(pitch_vec + interval_vec)

In [ ]:
motif_vectors_v1 = {
    motif: motif_vector_v1(songs, motif)
    for motif in {"ลาว", "เขมร", "แขก", "จีน", "พม่า"}  # filtered motifs
}

### V2 (13D)

- 7D **Pitch** (1–7): pitch-class usage (ด ร ม ฟ ซ ล ท)
- 3D **Interval** (8–10): stepwise / medium / large motion
- 3D **Direction** (11–13): up / down / same movement


In [ ]:
def motif_vector_v2(songs, motif):
    base = motif_vector_v1(songs, motif)
    dir_vec = directional_interval_stats(   # +3D
        [sym for s in songs if s["motif"] == motif for sym in s["symbol_sequence"]]
    )
    return np.concatenate([base, dir_vec])

In [ ]:
motif_vectors_v2 = {
    motif: motif_vector_v2(songs, motif)
    for motif in motif_vectors_v1
}

### V3 (14D)

- **Pitch** (1–7): pitch-class usage  
- **Interval** (8–10): stepwise / medium / large motion  
- **Direction** (11–13): up / down / same movement  
- **Interval entropy** (14): variability of interval usage


In [ ]:
def motif_vector_v3(songs, motif):
    # base V2
    base = motif_vector_v2(songs, motif)

    # interval entropy
    all_symbols = [
        sym
        for s in songs if s["motif"] == motif
        for sym in s["symbol_sequence"]
    ]
    ent = interval_entropy(all_symbols)

    return np.concatenate([base, [ent]])


In [ ]:
motif_vectors_v3 = {
    motif: motif_vector_v3(songs, motif)
    for motif in motif_vectors_v2
}

### V4 (15D)

- **Pitch** (1–7): pitch-class usage  
- **Interval** (8–10): stepwise / medium / large motion  
- **Direction** (11–13): up / down / same movement  
- **Interval entropy** (14): variability of interval usage  
- **Pitch-class entropy** (15): concentration vs spread of pitch usage


In [ ]:
def motif_vector_v4(songs, motif):
    base = motif_vector_v3(songs, motif)

    all_symbols = [
        sym
        for s in songs if s["motif"] == motif
        for sym in s["symbol_sequence"]
    ]
    p_ent = pitch_class_entropy(all_symbols)

    return np.concatenate([base, [p_ent]])

In [ ]:
motif_vectors_v4 = {
    motif: motif_vector_v4(songs, motif)
    for motif in motif_vectors_v3
}

### V5 (15 + K D)

- **Pitch** (1–7): pitch-class usage  
- **Interval** (8–10): stepwise / medium / large motion  
- **Direction** (11–13): up / down / same movement  
- **Interval entropy** (14): variability of interval usage  
- **Pitch-class entropy** (15): concentration of pitch usage  
- **Top-K pitch bigrams** (16–15+K): characteristic pitch transitions


In [ ]:
MOTIFS_USED = list(motif_vectors_v4.keys())
TOP_K = 25

TOP_K_BIGRAMS = get_top_k_bigrams(songs, MOTIFS_USED, K=TOP_K)
TOP_K_BIGRAMS

In [ ]:
def bigram_vector_from_symbols(symbols, top_k_bigrams):
    bigrams = extract_pitch_bigrams(symbols)
    counter = Counter(bigrams)
    total = sum(counter.values())

    return [
        counter.get(bg, 0) / total * 100 if total else 0
        for bg in top_k_bigrams
    ]

In [ ]:
def motif_vector_v5(songs, motif, top_k_bigrams):
    base = motif_vector_v4(songs, motif)

    all_symbols = [
        sym
        for s in songs if s["motif"] == motif
        for sym in s["symbol_sequence"]
    ]

    bg_vec = bigram_vector_from_symbols(all_symbols, top_k_bigrams)

    return np.concatenate([base, bg_vec])

In [ ]:
motif_vectors_v5 = {
    motif: motif_vector_v5(songs, motif, TOP_K_BIGRAMS)
    for motif in motif_vectors_v4
}

### Cosine Sim V1 to V5 motifs to motifs


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# ===== Motif-level cosine similarity (V2) =====

motifs = list(motif_vectors_v2.keys())
X = np.vstack([motif_vectors_v2[m] for m in motifs])

sim_matrix = cosine_similarity(X)

df_similarity_v2 = pd.DataFrame(
    sim_matrix,
    index=motifs,
    columns=motifs
)

df_similarity_v2

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 5))
plt.imshow(df_similarity_v2.values, cmap="viridis")
plt.colorbar(label="Cosine Similarity")

plt.xticks(range(len(motifs)), motifs, rotation=45)
plt.yticks(range(len(motifs)), motifs)

plt.title("Motif-level Cosine Similarity (V2)")
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# ===== Motif-level cosine similarity (V2) =====

motifs = list(motif_vectors_v5.keys())
X = np.vstack([motif_vectors_v5[m] for m in motifs])

sim_matrix = cosine_similarity(X)

df_similarity_v5 = pd.DataFrame(
    sim_matrix,
    index=motifs,
    columns=motifs
)

df_similarity_v5

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 5))
plt.imshow(df_similarity_v5.values, cmap="viridis")
plt.colorbar(label="Cosine Similarity")

plt.xticks(range(len(motifs)), motifs, rotation=45)
plt.yticks(range(len(motifs)), motifs)

plt.title("Motif-level Cosine Similarity (V5)")
plt.tight_layout()
plt.show()

### Fragment Vectorization (for testing)


In [ ]:
def get_fragment_symbols(song, n_symbols=16):
    return song["symbol_sequence"][:n_symbols]

def fragment_vector_v1(symbols):
    return np.array(
        pitch_vector_from_symbols(symbols)
        + interval_vector_from_symbols(symbols)
    )

def fragment_vector_v2(symbols):
    return np.concatenate([
        fragment_vector_v1(symbols),
        directional_interval_stats(symbols)
    ])

def fragment_vector_v3(symbols):
    base = fragment_vector_v2(symbols)
    ent = interval_entropy(symbols)
    return np.concatenate([base, [ent]])

def fragment_vector_v4(symbols):
    base = fragment_vector_v3(symbols)
    p_ent = pitch_class_entropy(symbols)
    return np.concatenate([base, [p_ent]])

def fragment_vector_v5(symbols, top_k_bigrams):
    base = fragment_vector_v4(symbols)
    bg_vec = bigram_vector_from_symbols(symbols, top_k_bigrams)
    return np.concatenate([base, bg_vec])

### Cosine Similarity demo with fragment






In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def rank_fragment(fragment_vec, motif_vectors):
    return sorted(
        {
            motif: cosine_similarity(
                fragment_vec.reshape(1, -1),
                vec.reshape(1, -1)
            )[0, 0]
            for motif, vec in motif_vectors.items()
        }.items(),
        key=lambda x: x[1],
        reverse=True
    )

In [ ]:
raw_frag = [ "--ดํล", "ดํลรํดํ", "--ฟร", "ฟรซฟ", "--ดํล", "--รํดํ", "ฟรซฟ", "ลซดํล" ] #khmer เขมร
frag = extract_symbols(raw_frag, strip_octave=True)

In [ ]:
raw_frag = [ "ดํรํดํล", "ดํซลดํ", "ดํรํดํล", "ดํลซฟ", "----", "ดรฟซ", "ลฟลซ", "-ฟ-ร" ] #khmer เขมร
frag = extract_symbols(raw_frag, strip_octave=True)


In [ ]:
raw_frag = [ "----", "----", "-ซ-ล", "-ท-ดํ", "---ท", "--ดํรํ", "มํรํดํท", "ดํรํ-ดํ" ] #แขก
frag = extract_symbols(raw_frag, strip_octave=True)

In [ ]:
raw_frag = [ "----", "----", "--ซล", "ดํมรด", "รม-ซ", "-ล--", "ดํลซม", "ซล-ดํ","รํมํรํดํ" ] #ลาว correct
frag = extract_symbols(raw_frag, strip_octave=True)

In [ ]:
raw_frag = [
          "----",
          "รมซล",
          "ซลดํล",
          "-ซซซ",
          "-ดรม",
          "-ซ-ล",
          "-ดํ-มํ",
          "-รํมํซํ"
        ] #ลาว  correct
frag = extract_symbols(raw_frag, strip_octave=True)

In [ ]:
raw_frag =[
          "ดํลซม",
          "-ร-ด",
          "--มร",
          "ดร-ม",
          "รด-ร",
          "-ม-ซ",
          "ลซดํล",
          "ซม-ร"
        ] #ลาว expected wrong

frag = extract_symbols(raw_frag, strip_octave=True)

In [ ]:
raw_frag = [
          "---ฟ",
          "--ลดํ",
          "-ดํ-รํ",
          "ดํดํดํดํ",
          "-ฟํ-ล",
          "ดํดํดํดํ",
          "-ล-รํ",
          "ดํดํดํดํ"
        ]  #ลาว expected wrong

frag = extract_symbols(raw_frag, strip_octave=True)

In [ ]:
raw_frag = [
          "-มํ-รํ",
          "-ดํรํมํ",
          "--ดํรํ",
          "มํรํดํล",
          "--ดํล",
          "ซมซล",
          "-ดํรํมํ",
          "-รํ-ดํ"
        ]
frag = extract_symbols(raw_frag, strip_octave=True)

In [ ]:
raw_frag = ["----","มดรม","รมฟซ","ฟซทล","-ลล-","ซฟมฟ","-ฟฟ-","มรดร"]
frag = extract_symbols(raw_frag, strip_octave=True)

In [ ]:
raw_frag = [ "-รซล", "ทดํรํ", "--ดํมํ", "รํดํทล", "ทลซดํ", "-ท-ล", "ทลซล", "ซฟมฟ" ] #khaek
frag = extract_symbols(raw_frag, strip_octave=True)

In [ ]:
raw_frag = [ "----", "-มํรํดํ", "--ลซ", "มซลดํ", "-ซลซ", "ลดํ-ล", "ดํลซม", "-รมซ" ]
frag = extract_symbols(raw_frag, strip_octave=True)

In [ ]:
rank_fragment(fragment_vector_v1(frag), motif_vectors_v1)


In [ ]:
rank_fragment(fragment_vector_v2(frag), motif_vectors_v2)

In [ ]:
rank_fragment(fragment_vector_v3(frag), motif_vectors_v3)

In [ ]:
rank_fragment(fragment_vector_v4(frag), motif_vectors_v4)

In [ ]:
rank_fragment(fragment_vector_v5(frag, TOP_K_BIGRAMS), motif_vectors_v5)

## Fragment to motifs Accuracy


### V1 to V5 vector with motifs accuracy


	•	Fragment size = 32 symbols (1 bar)
	•	Ignore fragments with >50% rests
	•	Evaluate:
	•	Cosine (V1–V5)
	•	LogReg (V1–V5)


In [ ]:
def extract_non_overlapping_fragments(
    songs,
    window_size=32,
    rest_threshold=0.5,
    strip_octave=True
):
    fragments = []

    for s in songs:
        motif = s["motif"]
        raw_seq = s["sequence"]

        # Convert to pitch symbols (remove rests automatically)
        symbols = extract_symbols(raw_seq, strip_octave=strip_octave)

        # Non-overlapping windows
        for i in range(0, len(raw_seq), window_size):
            window_tokens = raw_seq[i:i+window_size]

            if len(window_tokens) < window_size:
                continue

            # Rest ratio check
            rest_count = sum(1 for t in window_tokens if t == "----")
            if rest_count / window_size > rest_threshold:
                continue

            # Convert to symbols
            frag_symbols = extract_symbols(window_tokens, strip_octave=strip_octave)

            if len(frag_symbols) < 5:  # safety guard
                continue

            fragments.append((frag_symbols, motif))

    return fragments

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import pandas as pd

def fragment_cosine_evaluation(
    fragments,
    motif_vectors,
    fragment_vector_fn,
    extra_args=None
):
    y_true, y_pred = [], []

    for symbols, true_motif in fragments:

        if extra_args:
            x = fragment_vector_fn(symbols, *extra_args)
        else:
            x = fragment_vector_fn(symbols)

        sims = {
            m: cosine_similarity(
                x.reshape(1, -1),
                vec.reshape(1, -1)
            )[0, 0]
            for m, vec in motif_vectors.items()
        }

        pred = max(sims.items(), key=lambda x: x[1])[0]

        y_true.append(true_motif)
        y_pred.append(pred)

    labels = sorted(set(y_true))
    acc = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    return round(acc, 4), pd.DataFrame(cm, index=labels, columns=labels)

In [ ]:
MOTIFS_USED = ["ลาว", "เขมร", "แขก"]

fragments = extract_non_overlapping_fragments(
    songs,
    window_size=32,
    rest_threshold=0.5
)

# V1
acc_v1_frag, cm_v1_frag = fragment_cosine_evaluation(
    fragments,
    motif_vectors_v1,
    fragment_vector_v1
)

# V2
acc_v2_frag, cm_v2_frag = fragment_cosine_evaluation(
    fragments,
    motif_vectors_v2,
    fragment_vector_v2
)

# V3
acc_v3_frag, cm_v3_frag = fragment_cosine_evaluation(
    fragments,
    motif_vectors_v3,
    fragment_vector_v3
)

# V4
acc_v4_frag, cm_v4_frag = fragment_cosine_evaluation(
    fragments,
    motif_vectors_v4,
    fragment_vector_v4
)

# V5
acc_v5_frag, cm_v5_frag = fragment_cosine_evaluation(
    fragments,
    motif_vectors_v5,
    fragment_vector_v5,
    extra_args=[TOP_K_BIGRAMS]
)

acc_v1_frag, acc_v2_frag, acc_v3_frag, acc_v4_frag, acc_v5_frag

### LogReg


In [ ]:
fragments = extract_non_overlapping_fragments(
    songs,
    window_size=32,
    rest_threshold=0.5,
    strip_octave=True
)

In [ ]:
for i, (symbols, motif) in enumerate(fragments[:5]):
    print(f"Fragment {i} | Motif: {motif}")
    print(symbols)
    print("-" * 40)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import pandas as pd

def fragment_logreg_evaluation(
    fragments,
    fragment_vector_fn
):
    y_true, y_pred = [], []

    for i, (test_symbols, test_label) in enumerate(fragments):

        # --- split ---
        train_data = [
            (symbols, label)
            for j, (symbols, label) in enumerate(fragments)
            if j != i
        ]

        # --- build train ---
        X_train = np.vstack([
            fragment_vector_fn(symbols)
            for symbols, _ in train_data
        ])
        y_train = [label for _, label in train_data]

        # --- build test ---
        X_test = fragment_vector_fn(test_symbols).reshape(1, -1)

        # --- train ---
        clf = LogisticRegression(
            solver="lbfgs",
            max_iter=1000
        )
        clf.fit(X_train, y_train)

        # --- predict ---
        pred = clf.predict(X_test)[0]

        y_true.append(test_label)
        y_pred.append(pred)

    labels = sorted(set(y_true))
    acc = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    return round(acc, 4), pd.DataFrame(cm, index=labels, columns=labels)

In [ ]:
acc_f_v1, cm_f_v1 = fragment_logreg_evaluation(
    fragments,
    fragment_vector_v1
)

acc_f_v2, cm_f_v2 = fragment_logreg_evaluation(
    fragments,
    fragment_vector_v2
)

acc_f_v3, cm_f_v3 = fragment_logreg_evaluation(
    fragments,
    fragment_vector_v3
)

acc_f_v4, cm_f_v4 = fragment_logreg_evaluation(
    fragments,
    fragment_vector_v4
)

acc_f_v5, cm_f_v5 = fragment_logreg_evaluation(
    fragments,
    lambda s: fragment_vector_v5(s, TOP_K_BIGRAMS)
)

acc_f_v1, acc_f_v2, acc_f_v3, acc_f_v4, acc_f_v5

## Song to motifs Accuracy


**LOSO = Leave-One-Song-Out.**

	•	Hold one song out as test
	•	Train on all remaining songs
	•	Predict the held-out song
	•	Repeat for every song
	•	Average the results


In [ ]:
# ===== Song-level vector wrappers =====

def song_vector_v1(song):
    return motif_vector_v1([song], song["motif"])

def song_vector_v2(song):
    return motif_vector_v2([song], song["motif"])

def song_vector_v3(song):
    return motif_vector_v3([song], song["motif"])

def song_vector_v4(song):
    return motif_vector_v4([song], song["motif"])

def song_vector_v5(song, top_k_bigrams):
    return motif_vector_v5([song], song["motif"], top_k_bigrams)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
import pandas as pd

def loso_cosine_evaluation(
    songs,
    song_vector_fn,
    motifs_used,
    extra_args=None
):
    y_true, y_pred = [], []

    for i, test_song in enumerate(songs):
        if test_song["motif"] not in motifs_used:
            continue

        # split
        train_songs = [
            s for j, s in enumerate(songs)
            if j != i and s["motif"] in motifs_used
        ]

        # build motif centroids from TRAIN only
        centroids = {}
        for m in motifs_used:
            vecs = []
            for s in train_songs:
                if s["motif"] == m:
                    if extra_args:
                        vecs.append(song_vector_fn(s, *extra_args))
                    else:
                        vecs.append(song_vector_fn(s))
            centroids[m] = np.mean(np.vstack(vecs), axis=0)

        # test vector
        if extra_args:
            x_test = song_vector_fn(test_song, *extra_args)
        else:
            x_test = song_vector_fn(test_song)

        # cosine similarity
        sims = {
            m: cosine_similarity(
                x_test.reshape(1, -1),
                centroids[m].reshape(1, -1)
            )[0, 0]
            for m in centroids
        }

        pred = max(sims.items(), key=lambda x: x[1])[0]

        y_true.append(test_song["motif"])
        y_pred.append(pred)

    labels = sorted(set(y_true))
    # acc = accuracy_score(y_true, y_pred)
    acc = round(accuracy_score(y_true, y_pred), 4)
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    return acc, pd.DataFrame(cm, index=labels, columns=labels)

---


In [ ]:
MOTIFS_EVAL = ["ลาว", "เขมร", "แขก"]

In [ ]:
acc_v1, cm_v1 = loso_cosine_evaluation(
    songs,
    song_vector_v1,
    MOTIFS_EVAL
)
acc_v1, cm_v1

In [ ]:
acc_v2, cm_v2 = loso_cosine_evaluation(
    songs,
    song_vector_v2,
    MOTIFS_EVAL
)
acc_v2, cm_v2

In [ ]:
acc_v3, cm_v3 = loso_cosine_evaluation(
    songs,
    song_vector_v3,
    MOTIFS_EVAL
)
acc_v3, cm_v3

In [ ]:
acc_v4, cm_v4 = loso_cosine_evaluation(
    songs,
    song_vector_v4,
    MOTIFS_EVAL
)
acc_v4, cm_v4

In [ ]:
acc_v5, cm_v5 = loso_cosine_evaluation(
    songs,
    song_vector_v5,
    MOTIFS_EVAL,
    extra_args=[TOP_K_BIGRAMS]
)
acc_v5, cm_v5

### LogReg


### Baseline Classification Setup (Logistic Regression)

**Unit of analysis:**  
- Whole song

**Label:**  
- Motif (e.g. ลาว, เขมร, แขก)

**Evaluation protocol:**  
- LOSO (Leave-One-Song-Out)


For Logistic Regression:

- **X**: matrix of song-level feature vectors  
  - Shape: *(num_songs, feature_dim)*

- **y**: motif labels  
  - Shape: *(num_songs,)*

#### LOSO evaluation procedure

For each iteration:

1. Hold **one song** out as the test sample  
2. Train the model on **all remaining songs**  
3. Predict the motif of the held-out song  
4. Repeat until every song has been used as test once  

**Final metrics:**  
- Overall accuracy  
- Confusion matrix



In [ ]:
def song_vector_v1(song):
    symbols = song["symbol_sequence"]
    return np.array(
        pitch_vector_from_symbols(symbols)
        + interval_vector_from_symbols(symbols)
    )

def song_vector_v2(song):
    symbols = song["symbol_sequence"]
    return np.concatenate([
        song_vector_v1(song),
        directional_interval_stats(symbols)     # +3D
    ])

def song_vector_v3(song):
    symbols = song["symbol_sequence"]
    return np.concatenate([
        song_vector_v2(song),
        [interval_entropy(symbols)]              # +1D
    ])

def song_vector_v4(song):
    symbols = song["symbol_sequence"]
    return np.concatenate([
        song_vector_v3(song),
        [pitch_class_entropy(symbols)]           # +1D
    ])

def song_vector_v5(song, top_k_bigrams):
    symbols = song["symbol_sequence"]
    return np.concatenate([
        song_vector_v4(song),
        bigram_vector_from_symbols(symbols, top_k_bigrams)
    ])


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import pandas as pd

def loso_logreg_evaluation(
    songs,
    song_vector_fn,
    motifs_used
):
    y_true, y_pred = [], []

    for i, test_song in enumerate(songs):
        if test_song["motif"] not in motifs_used:
            continue

        # --- split ---
        train_songs = [
            s for j, s in enumerate(songs)
            if j != i and s["motif"] in motifs_used
        ]

        # --- build train data ---
        X_train = np.vstack([song_vector_fn(s) for s in train_songs])
        y_train = [s["motif"] for s in train_songs]

        # --- build test sample ---
        X_test = song_vector_fn(test_song).reshape(1, -1)

        # --- train ---
        clf = LogisticRegression(
            multi_class="auto",
            solver="lbfgs",
            max_iter=1000
        )
        clf.fit(X_train, y_train)

        # --- predict ---
        pred = clf.predict(X_test)[0]

        y_true.append(test_song["motif"])
        y_pred.append(pred)

    labels = sorted(set(y_true))
    acc = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    return round(acc, 4), pd.DataFrame(cm, index=labels, columns=labels)

In [ ]:
MOTIFS_USED = ["ลาว", "เขมร", "แขก"]

acc_v1, cm_v1 = loso_logreg_evaluation(
    songs,
    song_vector_v1,
    MOTIFS_USED
)

acc_v1, cm_v1

In [ ]:
acc_v2, cm_v2 = loso_logreg_evaluation(songs, song_vector_v2, MOTIFS_USED)
acc_v3, cm_v3 = loso_logreg_evaluation(songs, song_vector_v3, MOTIFS_USED)
acc_v4, cm_v4 = loso_logreg_evaluation(songs, song_vector_v4, MOTIFS_USED)

acc_v5, cm_v5 = loso_logreg_evaluation(
    songs,
    lambda s: song_vector_v5(s, TOP_K_BIGRAMS),
    MOTIFS_USED
)

In [ ]:
df_acc = pd.DataFrame([
    ("V1", acc_v1),
    ("V2", acc_v2),
    ("V3", acc_v3),
    ("V4", acc_v4),
    ("V5", acc_v5),
], columns=["Version", "LOSO Accuracy"])

df_acc

	•	V1 strong baseline (0.625) → pitch + interval already capture most signal
	•	V2–V4 drop slightly → added features introduce noise under very small data
	•	V5 improves best (0.6875) → order information (bigrams) actually helps

  More features ≠ better performance under low-data; sequence information matters more than higher-order statistics.
